In [2]:
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split


In [7]:
os.listdir('image/squat')

['.DS_Store', 'squat', 'plank']

In [1]:
def load_data_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (150, 150))  # Resize to consistent size
            images.append(img)
            labels.append(label)
    return images, labels

squat_images, squat_labels = load_data_from_folder('squat', 0)
plank_images, plank_labels = load_data_from_folder('plank', 1)

X = np.array(squat_images + plank_images)
y = np.array(squat_labels + plank_labels)

# Normalize the data
X = X / 255.0

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


NameError: name 'os' is not defined

In [9]:
model = Sequential()

# First convolution layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the layers
model.add(Flatten())

# Dense layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [10]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
39/39 [==============================] - 9s 228ms/step - loss: 0.1936 - accuracy: 0.9421 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 2/10
39/39 [==============================] - 9s 228ms/step - loss: 0.0031 - accuracy: 0.9992 - val_loss: 3.5511e-05 - val_accuracy: 1.0000
Epoch 3/10
39/39 [==============================] - 9s 222ms/step - loss: 4.2889e-04 - accuracy: 1.0000 - val_loss: 9.6106e-07 - val_accuracy: 1.0000
Epoch 4/10
39/39 [==============================] - 9s 230ms/step - loss: 2.2235e-05 - accuracy: 1.0000 - val_loss: 1.2459e-06 - val_accuracy: 1.0000
Epoch 5/10
39/39 [==============================] - 9s 224ms/step - loss: 6.2121e-06 - accuracy: 1.0000 - val_loss: 8.2959e-07 - val_accuracy: 1.0000
Epoch 6/10
39/39 [==============================] - 9s 236ms/step - loss: 1.1239e-05 - accuracy: 1.0000 - val_loss: 3.4289e-07 - val_accuracy: 1.0000
Epoch 7/10
39/39 [==============================] - 10s 254ms/step - loss: 2.6218e-06 - accuracy: 1.0000 - val_l

In [11]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


10/10 [==============================] - 1s 61ms/step - loss: 1.3959e-07 - accuracy: 1.0000
Test Accuracy: 100.00%


In [12]:
model.save('squat_vs_plank_model.h5')


/Users/jun1/Library/CloudStorage/OneDrive-Personal/Jun/핀테크/Deep_learning_example/.conda/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
import cv2
import numpy as np
import tensorflow as tf


In [8]:
model = tf.keras.models.load_model('/Users/jun1/Library/CloudStorage/OneDrive-Personal/Jun/핀테크/기계학습과딥러닝_이재욱/Squats-angle-detection-using-OpenCV-and-mediapipe_v1-main/squat_classifier_model.h5')


In [13]:
def predict_activity(frame, model):
    # Resize the frame to the same size as the training data
    frame_resized = cv2.resize(frame, (150, 150))
    frame_normalized = frame_resized / 255.0  # normalize the frame
    frame_flattened = frame_normalized.flatten()[:132]  # flatten and take first 132 values
    frame_reshaped = np.reshape(frame_flattened, (1, 132))
    
    # Make prediction
    prediction = model.predict(frame_reshaped)
    
    if prediction > 0.5:
        return "Plank"
    else:
        return "Squat"


# Capture video from default camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()  # Read a frame from the video capture
    
    if not ret:
        break

    activity = predict_activity(frame, model)
    
    # Display the predicted activity on the video frame
    cv2.putText(frame, activity, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('Activity Detection', frame)

    # Press 'q' to quit the video window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 8ms/step


: 

In [14]:
cap = cv2.VideoCapture(1)
import cv2
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('squat_vs_plank_model.h5')

def predict_activity(frame, model):
    # Resize the frame to the same size as the training data
    frame_resized = cv2.resize(frame, (150, 150))
    frame_normalized = frame_resized / 255.0  # normalize the frame
    frame_reshaped = np.reshape(frame_normalized, (1, 150, 150, 3))
    
    # Make prediction
    prediction = model.predict(frame_reshaped)
    
    if prediction > 0.5:
        return "Plank"
    else:
        return "Squat"

# Open the MP4 file
cap = cv2.VideoCapture('/Users/jun1/Library/CloudStorage/OneDrive-Personal/Jun/핀테크/기계학습과딥러닝_이재욱/Squats-angle-detection-using-OpenCV-and-mediapipe_v1-main/squat.mp4')

while True:
    ret, frame = cap.read()  # Read a frame from the video
    
    if not ret:
        break

    activity = predict_activity(frame, model)
    
    # Display the predicted activity on the video frame
    cv2.putText(frame, activity, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('Activity Detection', frame)

    # Press 'q' to quit the video window
    if cv2.waitKey(30) & 0xFF == ord('q'):  # Wait for a short while before moving to the next frame
        break

cap.release()
cv2.destroyAllWindows()


OpenCV: out device of bound (0-0): 1
OpenCV: camera failed to properly initialize!


1/1 [==============================] - 0s 14ms/step
